# 원문-번역문 의미 기반 구 단위 매칭 파이프라인

이 노트북은 한문 원문과 번역문 간의 구 단위 정렬을 위한 파이프라인을 구현합니다. 다음과 같은 주요 기능을 포함합니다:

1. **텍스트 토크나이징**: 원문과 번역문을 의미 단위로 분할
2. **임베딩 계산**: 각 의미 단위에 대한 벡터 표현 생성
3. **구 단위 정렬**: 동적 프로그래밍 기반 정렬 알고리즘
4. **파일 입출력**: Excel 파일 처리

이 파이프라인은 원문-번역문 쌍을 입력으로 받아 구 단위로 정렬된 결과를 출력합니다.

## 1. 필요한 패키지 설치 및 임포트

아래 셀을 실행하여 필요한 패키지를 설치하고 임포트합니다.

In [ ]:
# Install necessary packages (GPU version with CUDA 12.1)
%pip install regex pandas numpy tqdm openpyxl konlpy
%pip install torch>=2.6.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
%pip install sentence-transformers FlagEmbedding

In [ ]:
# Import necessary libraries
import os
import sys
import logging
import torch
import regex as re
import numpy as np
import pandas as pd
from typing import List, Tuple, Dict, Any, Optional, Callable
from tqdm.notebook import tqdm
from konlpy.tag import Kkma
from FlagEmbedding import BGEM3FlagModel
import sentencepiece as spm
from tokenizers import SentencePieceBPETokenizer
from scipy.optimize import linear_sum_assignment
from transformers import XLMRobertaTokenizerFast

# 환경 변수 설정
os.environ['PYTORCH_DISABLE_TORCH_LOAD_SECURITY_CHECK'] = '1'

# 한국어 형태소 분석기
tokenizer = Kkma()

# 모델 로딩 with 다중 fallback
def load_embedding_model():
    """여러 옵션을 시도하여 임베딩 모델 로드"""
    
    # Option 1: BGE 모델
    try:
        from FlagEmbedding import BGEM3FlagModel
        model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)
        print("✅ BGE model loaded successfully")
        return model, 'bge'
    except Exception as e:
        print(f"❌ BGE model failed: {e}")
    
    # Option 2: SentenceTransformers
    try:
        from sentence_transformers import SentenceTransformer
        model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
        print("✅ SentenceTransformer model loaded successfully")
        return model, 'sentence_transformer'
    except Exception as e:
        print(f"❌ SentenceTransformer failed: {e}")
    
    # Option 3: 더미 모델
    print("🔄 Using dummy embeddings")
    return None, 'dummy'

# 모델 로드
model, model_type = load_embedding_model()

# 범용 임베딩 함수
def compute_embeddings(texts, model=model, model_type=model_type):
    """모델 타입에 따른 임베딩 계산"""
    if model_type == 'bge':
        output = model.encode(texts, return_dense=True, return_sparse=False, return_colbert_vecs=False)
        return output['dense_vecs']
    elif model_type == 'sentence_transformer':
        return model.encode(texts, convert_to_numpy=True, normalize_embeddings=True)
    else:  # dummy
        embeddings = []
        for text in texts:
            import hashlib
            text_hash = hashlib.md5(str(text).encode()).hexdigest()
            seed = int(text_hash[:8], 16) % (2**31)
            np.random.seed(seed)
            dummy_emb = np.random.randn(384).astype(np.float32)
            dummy_emb = dummy_emb / (np.linalg.norm(dummy_emb) + 1e-8)
            embeddings.append(dummy_emb)
        return np.array(embeddings)

# Logging setup
logging.basicConfig(
    format="[%(levelname)s] %(asctime)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    level=logging.INFO,
)

print(f"✅ Setup complete. Using {model_type} embeddings.")

## 2. 임베딩 모듈 (embedder.py)

텍스트 구에 대한 임베딩을 계산하고 캐시를 관리하는 모듈입니다.

In [ ]:
import os
import logging
import numpy as np
import pandas as pd
from konlpy.tag import Kkma
from typing import List, Tuple, Dict, Any, Optional

# 로거 설정 (먼저 설정해야 함)
logging.basicConfig(
    format="[%(levelname)s] %(asctime)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    level=logging.INFO,
)
logger = logging.getLogger(__name__)

# 환경 변수 설정
os.environ['PYTORCH_DISABLE_TORCH_LOAD_SECURITY_CHECK'] = '1'

# 한국어 형태소 분석기
tokenizer = Kkma()

# 모델 로딩 with 다중 fallback
def load_embedding_model():
    """여러 옵션을 시도하여 임베딩 모델 로드"""
    
    # Option 1: BGE 모델
    try:
        from FlagEmbedding import BGEM3FlagModel
        model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)
        logger.info("✅ BGE model loaded successfully")
        return model, 'bge'
    except Exception as e:
        logger.warning(f"❌ BGE model failed: {e}")
    
    # Option 2: SentenceTransformers
    try:
        from sentence_transformers import SentenceTransformer
        model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
        logger.info("✅ SentenceTransformer model loaded successfully")
        return model, 'sentence_transformer'
    except Exception as e:
        logger.warning(f"❌ SentenceTransformer failed: {e}")
    
    # Option 3: 더미 모델
    logger.info("🔄 Using dummy embeddings")
    return None, 'dummy'

# 모델 로드
model, model_type = load_embedding_model()

# 범용 임베딩 함수
def compute_embeddings(texts, model=model, model_type=model_type):
    """모델 타입에 따른 임베딩 계산"""
    if model_type == 'bge':
        output = model.encode(texts, return_dense=True, return_sparse=False, return_colbert_vecs=False)
        return output['dense_vecs']
    elif model_type == 'sentence_transformer':
        return model.encode(texts, convert_to_numpy=True, normalize_embeddings=True)
    else:  # dummy
        embeddings = []
        for text in texts:
            import hashlib
            text_hash = hashlib.md5(str(text).encode()).hexdigest()
            seed = int(text_hash[:8], 16) % (2**31)
            np.random.seed(seed)
            dummy_emb = np.random.randn(384).astype(np.float32)
            dummy_emb = dummy_emb / (np.linalg.norm(dummy_emb) + 1e-8)
            embeddings.append(dummy_emb)
        return np.array(embeddings)

logger.info(f"✅ Setup complete. Using {model_type} embeddings.")

# 테스트
test_texts = ["테스트 문장", "또 다른 문장"]
test_embeddings = compute_embeddings(test_texts)
logger.info(f"Test embeddings shape: {test_embeddings.shape}")

## 3. 구두점 처리 모듈 (punctuation.py)

괄호 처리 및 마스킹 기능을 제공하는 모듈입니다.

In [ ]:
# Punctuation handling module
MASK_TEMPLATE = '[MASK{}]'

HALF_WIDTH_BRACKETS = [
    ('(', ')'),
    ('[', ']'),
]
FULL_WIDTH_BRACKETS = [
    ('（', '）'),
    ('［', '］'),
]
TRANS_BRACKETS = [
    ('<', '>'),
    ('《', '》'),
    ('〈', '〉'),
    ('「', '」'),
    ('『', '』'),
    ('〔', '〕'),
    ('【', '】'),
    ('〖', '〗'),
    ('〘', '〙'),
    ('〚', '〛'),
]

ALL_BRACKETS = HALF_WIDTH_BRACKETS + FULL_WIDTH_BRACKETS + TRANS_BRACKETS

def mask_brackets(text: str, text_type: str) -> Tuple[str, List[str]]:
    """Mask content within brackets according to rules."""
    assert text_type in {'source', 'target'}, "text_type must be 'source' or 'target'"

    masks: List[str] = []
    mask_id = [0]

    def safe_sub(pattern, repl, s):
        def safe_replacer(m):
            if '[MASK' in m.group(0):
                return m.group(0)
            return repl(m)
        return pattern.sub(safe_replacer, s)

    patterns: List[Tuple[re.Pattern, bool]] = []

    if text_type == 'source':
        for left, right in HALF_WIDTH_BRACKETS:
            patterns.append((re.compile(re.escape(left) + r'[^' + re.escape(left + right) + r']*?' + re.escape(right)), True))
        for left, right in FULL_WIDTH_BRACKETS:
            patterns.append((re.compile(re.escape(left)), False))
            patterns.append((re.compile(re.escape(right)), False))
    elif text_type == 'target':
        for left, right in HALF_WIDTH_BRACKETS + FULL_WIDTH_BRACKETS:
            patterns.append((re.compile(re.escape(left) + r'[^' + re.escape(left + right) + r']*?' + re.escape(right)), True))
        for left, right in TRANS_BRACKETS:
            patterns.append((re.compile(re.escape(left)), False))
            patterns.append((re.compile(re.escape(right)), False))

    def mask_content(s: str, pattern: re.Pattern, content_mask: bool) -> str:
        def replacer(match: re.Match) -> str:
            token = MASK_TEMPLATE.format(mask_id[0])
            masks.append(match.group())
            mask_id[0] += 1
            return token
        return safe_sub(pattern, replacer, s)

    for pattern, content_mask in patterns:
        if content_mask:
            text = mask_content(text, pattern, content_mask)
    for pattern, content_mask in patterns:
        if not content_mask:
            text = mask_content(text, pattern, content_mask)

    return text, masks

def restore_masks(text: str, masks: List[str]) -> str:
    """Restore masked tokens to their original content."""
    for i, original in enumerate(masks):
        text = text.replace(MASK_TEMPLATE.format(i), original)
    return text

## 4. 토크나이저 모듈 (tokenizer.py)

원문과 번역문을 의미 단위로 분할하는 모듈입니다.

In [ ]:
"""원문과 번역문을 의미 단위로 분할하는 모듈"""

import logging
import numpy as np
import regex
import re
from typing import List, Callable
from konlpy.tag import Kkma

# Kkma 형태소 분석기
kkma = Kkma()

# 미리 컴파일된 정규식
hanja_re    = regex.compile(r'\p{Han}+')
hangul_re   = regex.compile(r'^\p{Hangul}+$')
combined_re = regex.compile(
    r'(\p{Han}+)+(?:\p{Hangul}+)(?:은|는|이|가|을|를|에|에서|으로|로|와|과|도|만|으며|고|하고|의|때)?'
)

def split_src_meaning_units(text: str) -> list[str]:
    text = text.replace('\n', ' ').replace('：', '： ')
    tokens = regex.findall(r'\S+', text)
    units: list[str] = []
    i = 0

    while i < len(tokens):
        tok = tokens[i]
        m = combined_re.match(tok)
        if m:
            units.append(m.group(0))
            i += 1
            continue

        if hanja_re.search(tok):
            unit = tok
            j = i + 1
            while j < len(tokens) and hangul_re.match(tokens[j]):
                unit += tokens[j]
                j += 1
            units.append(unit)
            i = j
            continue

        if hangul_re.match(tok):
            korean_tokens = kkma.morphs(tok)
            units.extend(korean_tokens)
            i += 1
            continue

        units.append(tok)
        i += 1

    return units

def split_inside_chunk(chunk: str) -> list[str]:
    """
    조사, 어미, 그리고 '：' 기준으로 의미 단위 분할
    원형 보존, 공백 삽입 없이 분리
    """
    delimiters = ['을', '를', '이', '가', '은', '는', '에', '에서', '로', '으로',
                  '와', '과', '고', '며', '하고', '때', '의', '도', '만', '：']
    
    # lookbehind 패턴 생성
    pattern = '|'.join([f'(?<={re.escape(d)})' for d in delimiters])
    try:
        parts = re.split(pattern, chunk)
        return [p.strip() for p in parts if p.strip()]
    except:
        return [p for p in chunk.split() if p.strip()]

def find_target_span_end_simple(src_unit: str, remaining_tgt: str) -> int:
    hanja_chars = regex.findall(r'\p{Han}+', src_unit)
    if not hanja_chars:
        return 0
    last = hanja_chars[-1]
    idx = remaining_tgt.rfind(last)
    if idx == -1:
        return len(remaining_tgt)
    end = idx + len(last)
    next_space = remaining_tgt.find(' ', end)
    return next_space + 1 if next_space != -1 else len(remaining_tgt)

def find_target_span_end_semantic(
    src_unit: str,
    remaining_tgt: str,
    embed_func=compute_embeddings_with_cache,
    min_tokens: int = 1,
    max_tokens: int = 50,
    similarity_threshold: float = 0.4
) -> int:
    """최적화된 타겟 스팬 탐색 함수"""
    # 예외 처리 강화
    if not src_unit or not remaining_tgt:
        return 0
        
    try:
        # 1) 원문 임베딩 (단일 계산)
        src_emb = embed_func([src_unit])[0]
        
        # 2) 번역문 토큰 분리 및 누적 길이 계산
        tgt_tokens = remaining_tgt.split()
        if not tgt_tokens:
            return 0
            
        upper = min(len(tgt_tokens), max_tokens)
        cumulative_lengths = [0]
        current_length = 0
        
        for tok in tgt_tokens:
            current_length += len(tok) + 1  # 토큰 + 공백
            cumulative_lengths.append(current_length)
            
        # 3) 후보 세그먼트 생성 (메모리 효율적 방식)
        candidates = []
        candidate_indices = []
        
        # 적은 수의 후보만 생성하여 효율성 향상
        step_size = 1 if upper <= 10 else 2  # 토큰 10개 이하면 모든 후보 검사, 이상이면 2칸씩
        
        for end_i in range(min_tokens-1, upper, step_size):
            cand = " ".join(tgt_tokens[:end_i+1])
            candidates.append(cand)
            candidate_indices.append(end_i)
            
        # 4) 배치 임베딩 (한 번에 계산)
        cand_embs = embed_func(candidates)
        
        # 5) 최적 매칭 탐색 (코사인 유사도 + 길이 패널티)
        best_score = -1.0
        best_end_idx = cumulative_lengths[-1]  # 기본값은 전체 길이
        
        for i, emb in enumerate(cand_embs):
            # 코사인 유사도 계산
            score = np.dot(src_emb, emb) / (np.linalg.norm(src_emb) * np.linalg.norm(emb) + 1e-8)
            
            # 길이 패널티 (너무 짧은 매칭 방지)
            end_i = candidate_indices[i]
            length_ratio = (end_i + 1) / len(tgt_tokens)
            length_penalty = min(1.0, length_ratio * 2)  # 최대 1.0
            
            adjusted_score = score * length_penalty
            
            # 임계값 확인 및 최대값 갱신
            if adjusted_score > best_score and score >= similarity_threshold:
                best_score = adjusted_score
                best_end_idx = cumulative_lengths[end_i + 1]
                
        return best_end_idx
        
    except Exception as e:
        logger.warning(f"의미 매칭 오류, 단순 매칭으로 대체: {e}")
        return find_target_span_end_simple(src_unit, remaining_tgt)


def split_tgt_by_src_units(src_units: list[str], tgt_text: str) -> list[str]:
    results = []
    cursor = 0
    total = len(tgt_text)
    for src_u in src_units:
        remaining = tgt_text[cursor:]
        end_len = find_target_span_end_simple(src_u, remaining)
        chunk = tgt_text[cursor:cursor+end_len]
        results.extend(split_inside_chunk(chunk))
        cursor += end_len
    if cursor < total:
        results.extend(split_inside_chunk(tgt_text[cursor:]))
    return results

def split_tgt_by_src_units_semantic(src_units, tgt_text, embed_func=compute_embeddings_with_cache, min_tokens=1):
    tgt_tokens = tgt_text.split()
    N, T = len(src_units), len(tgt_tokens)
    if N == 0 or T == 0:
        return []

    dp = np.full((N+1, T+1), -np.inf)
    back = np.zeros((N+1, T+1), dtype=int)
    dp[0, 0] = 0.0

    # 원문 임베딩 계산
    src_embs = embed_func(src_units)

    # DP 테이블 채우기 (j 루프 범위 주의!)
    for i in range(1, N+1):
        for j in range(i*min_tokens, T-(N-i)*min_tokens+1):
            for k in range((i-1)*min_tokens, j-min_tokens+1):
                span = " ".join(tgt_tokens[k:j])
                tgt_emb = embed_func([span])[0]
                sim = float(np.dot(src_embs[i-1], tgt_emb)/((np.linalg.norm(src_embs[i-1])*np.linalg.norm(tgt_emb))+1e-8))
                score = dp[i-1, k] + sim
                if score > dp[i, j]:
                    dp[i, j] = score
                    back[i, j] = k

    # Traceback
    cuts = [T]
    curr = T
    for i in range(N, 0, -1):
        prev = int(back[i, curr])
        cuts.append(prev)
        curr = prev
    cuts = cuts[::-1]
    assert cuts[0] == 0 and cuts[-1] == T and len(cuts) == N + 1

    # Build actual spans
    tgt_spans = []
    for i in range(N):
        span = " ".join(tgt_tokens[cuts[i]:cuts[i+1]]).strip()
        tgt_spans.append(span)
    return tgt_spans

def split_tgt_meaning_units(
    src_text: str,
    tgt_text: str,
    use_semantic: bool = True,
    min_tokens: int = 1,
    max_tokens: int = 50
) -> list[str]:
    src_units = split_src_meaning_units(src_text)

    if use_semantic:
        return split_tgt_by_src_units_semantic(
            src_units,
            tgt_text,
            embed_func=compute_embeddings_with_cache,
            min_tokens=min_tokens
        )
    else:
        return split_tgt_by_src_units(src_units, tgt_text)

## 5. 정렬 모듈 (aligner.py)

원문과 번역문 구 간의 정렬을 위한 알고리즘입니다.

In [ ]:
# Aligner module
def cosine_similarity(vec1: Any, vec2: Any) -> float:
    """Calculate cosine similarity (handling zero vectors)."""
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    if norm1 == 0 or norm2 == 0:
        return 0.0
    return float(np.dot(vec1, vec2) / (norm1 * norm2))

def align_src_tgt(src_units, tgt_units, embed_func=compute_embeddings_with_cache):
    """Align source and target units."""
    logger.info(f"Source units: {len(src_units)} items, Target units: {len(tgt_units)} items")

    if len(src_units) != len(tgt_units):
        try:
            flatten_tgt = " ".join(tgt_units)
            new_tgt_units = split_tgt_by_src_units_semantic(src_units, flatten_tgt, embed_func, min_tokens=1)
            if len(new_tgt_units) == len(src_units):
                logger.info("Semantic re-alignment successful")
                return list(zip(src_units, new_tgt_units))
            else:
                logger.warning(f"Length mismatch after re-alignment: Source={len(src_units)}, Target={len(new_tgt_units)}")
        except Exception as e:
            logger.error(f"Error during semantic re-alignment: {e}")

        if len(src_units) > len(tgt_units):
            tgt_units.extend([""] * (len(src_units) - len(tgt_units)))
        else:
            src_units.extend([""] * (len(tgt_units) - len(src_units)))

    return list(zip(src_units, tgt_units))

def calculate_alignment_matrix(src_embs, tgt_embs, batch_size=512):
    """Optimized function for calculating large similarity matrices."""
    src_len, tgt_len = len(src_embs), len(tgt_embs)
    similarity_matrix = np.zeros((src_len, tgt_len))

    for i in range(0, src_len, batch_size):
        batch_src = src_embs[i:i + batch_size]
        for j in range(0, tgt_len, batch_size):
            batch_tgt = tgt_embs[j:j + batch_size]
            batch_src_norm = np.linalg.norm(batch_src, axis=1, keepdims=True)
            batch_tgt_norm = np.linalg.norm(batch_tgt, axis=1, keepdims=True)

            dots = np.matmul(batch_src, batch_tgt.T)
            norms = np.matmul(batch_src_norm, batch_tgt_norm.T)
            batch_sim = dots / (norms + 1e-8)

            similarity_matrix[i:i + batch_size, j:j + batch_size] = batch_sim

    return similarity_matrix

## 6. I/O 모듈 (io_manager.py)

Excel 파일을 읽고 처리한 후 결과를 저장하는 모듈입니다.

In [ ]:
# I/O module
def process_file(input_path: str, output_path: str, batch_size: int = 128, verbose: bool = False) -> None:
    """청크 단위로 최적화된 파일 처리 함수"""
    try:
        df = pd.read_excel(input_path, engine='openpyxl')
    except Exception as e:
        logger.error(f"[IO] Failed to read Excel file: {e}")
        return

    if '원문' not in df.columns or '번역문' not in df.columns:
        logger.error("[IO] Missing '원문' or '번역문' columns.")
        return

    outputs: List[Dict[str, Any]] = []
    total_rows = len(df)
    
    # 처리할 행을 작은 청크로 분할
    chunk_size = min(50, total_rows)  # 최대 50행씩 처리
    
    # 임베딩 캐시 초기화 - 메모리 관리
    global _embedding_cache
    
    for chunk_start in tqdm(range(0, total_rows, chunk_size), desc="Processing chunks"):
        chunk_end = min(chunk_start + chunk_size, total_rows)
        chunk_df = df.iloc[chunk_start:chunk_end]
        
        # 청크별 임베딩 캐시 관리 (메모리 효율성)
        if len(_embedding_cache) > 10000:  # 캐시가 너무 크면
            _embedding_cache = {}  # 초기화
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
        
        # 청크 내 행 처리
        for idx, row in enumerate(chunk_df.itertuples(index=False), start=chunk_start+1):
            src_text = str(getattr(row, '원문', '') or '')
            tgt_text = str(getattr(row, '번역문', '') or '')
            if verbose:
                print(f"\n[========= ROW {idx} =========]")
                print("Source (input):", src_text)
                print("Target (input):", tgt_text)

            try:
                masked_src, src_masks = mask_brackets(src_text, text_type="source")
                masked_tgt, tgt_masks = mask_brackets(tgt_text, text_type="target")

                src_units = split_src_meaning_units(masked_src)
                tgt_units = split_tgt_meaning_units(masked_src, masked_tgt, use_semantic=True, min_tokens=1)

                restored_src_units = [restore_masks(unit, src_masks) for unit in src_units]
                restored_tgt_units = [restore_masks(unit, tgt_masks) for unit in tgt_units]

                aligned_pairs = align_src_tgt(restored_src_units, restored_tgt_units, compute_embeddings_with_cache)
                aligned_src_units, aligned_tgt_units = zip(*aligned_pairs)

                if verbose:
                    print("Alignment result: (source to target comparison)")
                    for src_gu, tgt_gu in zip(aligned_src_units, aligned_tgt_units):
                        print(f"SRC: {src_gu} | TGT: {tgt_gu}")

                for gu_idx, (src_gu, tgt_gu) in enumerate(zip(aligned_src_units, aligned_tgt_units), start=1):
                    outputs.append({
                        "문장식별자": idx,
                        "구식별자": gu_idx,
                        "원문구": src_gu,
                        "번역구": tgt_gu,
                    })

            except Exception as e:
                logger.warning(f"[IO] Failed to process row {idx}: {e}")
                outputs.append({
                    "문장식별자": idx,
                    "구식별자": 1,
                    "원문구": src_text,
                    "번역구": tgt_text,
                })

    try:
        output_df = pd.DataFrame(outputs, columns=["문장식별자", "구식별자", "원문구", "번역구"])
        output_df.to_excel(output_path, index=False, engine='openpyxl')
        if verbose:
            logger.info(f"[IO] Results saved successfully: {output_path}")
    except Exception as e:
        logger.error(f"[IO] Failed to save results: {e}")
        

def process_file_parallel(input_path: str, output_path: str, num_workers: int = None, chunk_size: int = 20, gpu_strategy: str = "single") -> None:
    """
    병렬 처리를 사용한 파일 처리 함수 - GPU 최적화 버전
    
    Args:
        input_path: 입력 Excel 파일 경로
        output_path: 출력 Excel 파일 경로
        num_workers: 병렬 처리에 사용할 워커 수 (None이면 CPU 코어 수에 따라 자동 설정)
        chunk_size: 각 워커에게 할당할 행 수
        gpu_strategy: GPU 활용 전략 
            - "single": 첫 번째 워커만 GPU 사용 (안전)
            - "shared": 모든 워커가 GPU 공유 (고성능 GPU 필요)
            - "multi": 여러 GPU에 분산 (다중 GPU 환경)
            - "none": GPU 사용 안 함 (CPU만 사용)
    """
    import concurrent.futures
    import os
    import pandas as pd
    import torch
    from tqdm import tqdm
    import logging
    
    # 로거 사용 (기존 코드와 동일한 로거 사용)
    logger = logging.getLogger(__name__)
    
    try:
        # 파일 읽기 (기존 코드와 동일한 방식)
        try:
            df = pd.read_excel(input_path, engine='openpyxl')
        except Exception as e:
            logger.error(f"[IO] Failed to read Excel file: {e}")
            return

        if '원문' not in df.columns or '번역문' not in df.columns:
            logger.error("[IO] Missing '원문' or '번역문' columns.")
            return
        
        # 워커 수 결정
        if num_workers is None:
            cpu_cores = os.cpu_count() or 4
            if gpu_strategy == "multi" and torch.cuda.is_available():
                # 멀티 GPU 환경에서는 GPU 수에 맞게 조정
                gpu_count = torch.cuda.device_count()
                num_workers = max(min(cpu_cores, gpu_count * 2 if gpu_count > 0 else 4), 1)
            else:
                # 일반 환경에서는 CPU 코어 수 기준으로 설정
                num_workers = min(cpu_cores, 4)
        
        # GPU 설정 준비
        gpu_count = 0
        if torch.cuda.is_available():
            gpu_count = torch.cuda.device_count()
            logger.info(f"Available GPUs: {gpu_count}")
            # GPU 메모리 정리
            torch.cuda.empty_cache()
        else:
            logger.info("No GPU available, using CPU only")
            if gpu_strategy != "none":
                logger.warning("GPU strategy requested but no GPU available, falling back to CPU")
                gpu_strategy = "none"
        
        # 데이터 분할
        total_rows = len(df)
        row_indices = list(range(1, total_rows + 1))  # 1부터 시작하는 인덱스
        
        # 데이터 준비 - 작업량이 균등하도록 분배
        data_chunks = []
        for i in range(0, total_rows, chunk_size):
            chunk_indices = row_indices[i:i + chunk_size]
            chunk_data = []
            for idx in chunk_indices:
                row_idx = idx - 1  # 0-based 인덱스로 변환
                if row_idx < len(df):  # 인덱스 범위 체크
                    row = df.iloc[row_idx]
                    # None 체크 및 문자열 변환
                    src_text = str(row['원문']) if pd.notnull(row['원문']) else ""
                    tgt_text = str(row['번역문']) if pd.notnull(row['번역문']) else ""
                    
                    # 텍스트 길이 기반 가중치 계산
                    text_length = len(src_text) + len(tgt_text)
                    chunk_data.append((idx, src_text, tgt_text, text_length))
            
            if chunk_data:  # 빈 청크 방지
                data_chunks.append(chunk_data)
        
        # 청크별 작업량 계산 및 텍스트 길이 정보 제거
        processed_chunks = []
        for chunk in data_chunks:
            # 텍스트 길이 정보는 제거하고 원래 데이터만 유지
            processed_chunks.append([(item[0], item[1], item[2]) for item in chunk])
        
        # 작업량 기준으로 청크 정렬 (내림차순)
        chunks_with_size = [(i, sum(len(item[1])+len(item[2]) for item in chunk)) for i, chunk in enumerate(data_chunks)]
        chunks_with_size.sort(key=lambda x: x[1], reverse=True)
        sorted_chunk_indices = [i for i, _ in chunks_with_size]
        
        # GPU 할당 전략 설정
        if gpu_strategy == "single" and gpu_count > 0:
            # 첫 번째 워커만 GPU 사용
            gpu_assignments = [0 if i == 0 else -1 for i in range(num_workers)]
        elif gpu_strategy == "shared" and gpu_count > 0:
            # 모든 워커가 첫 번째 GPU 공유
            gpu_assignments = [0 for _ in range(num_workers)]
        elif gpu_strategy == "multi" and gpu_count > 1:
            # 다중 GPU에 워커 분산
            gpu_assignments = [i % gpu_count for i in range(num_workers)]
        else:
            # GPU 사용 안 함
            gpu_assignments = [-1 for _ in range(num_workers)]
        
        logger.info(f"Starting parallel processing with {num_workers} workers, {len(processed_chunks)} chunks")
        logger.info(f"GPU strategy: {gpu_strategy}, GPU assignments: {gpu_assignments}")
        
        # 청크와 워커 매핑 최적화 (긴 텍스트 = 강력한 GPU에 할당)
        chunk_worker_assignments = {}
        for i, chunk_idx in enumerate(sorted_chunk_indices):
            worker_idx = i % num_workers
            chunk_worker_assignments[chunk_idx] = worker_idx
        
        # 병렬 실행
        all_results = []
        with concurrent.futures.ProcessPoolExecutor(max_workers=num_workers) as executor:
            future_to_chunk = {}
            
            for chunk_idx, chunk_data in enumerate(processed_chunks):
                worker_idx = chunk_worker_assignments.get(chunk_idx, chunk_idx % num_workers)
                gpu_idx = gpu_assignments[worker_idx]
                
                # 워커별 메모리 사용량 조정
                memory_fraction = 0.8 if gpu_strategy == "shared" else 0.95
                
                future = executor.submit(
                    process_chunk_safe,  # 안전한 청크 처리 함수 사용
                    chunk_data,
                    gpu_idx=gpu_idx,
                    memory_fraction=memory_fraction
                )
                future_to_chunk[future] = chunk_idx
            
            # 결과 수집
            for future in tqdm(concurrent.futures.as_completed(future_to_chunk), 
                              total=len(future_to_chunk), 
                              desc="Processing chunks"):
                chunk_idx = future_to_chunk[future]
                try:
                    chunk_results = future.result()
                    if isinstance(chunk_results, list):  # 타입 체크 추가
                        all_results.extend(chunk_results)
                    else:
                        logger.error(f"Invalid result type from chunk {chunk_idx}: {type(chunk_results)}")
                except Exception as e:
                    logger.error(f"Error processing chunk {chunk_idx}: {e}")
        
        # 결과가 비어있는지 확인
        if not all_results:
            logger.error("No results were produced by parallel processing")
            return
            
        # 결과 정렬 (문장식별자, 구식별자 순)
        all_results.sort(key=lambda x: (x['문장식별자'], x['구식별자']))
        
        # 결과 저장
        try:
            output_df = pd.DataFrame(all_results, columns=["문장식별자", "구식별자", "원문구", "번역구"])
            output_df.to_excel(output_path, index=False, engine='openpyxl')
            logger.info(f"[IO] Results saved successfully: {output_path}")
        except Exception as e:
            logger.error(f"[IO] Failed to save results: {e}")
            
    except Exception as e:
        logger.error(f"Parallel processing failed: {e}")
        raise
    

def process_chunk_safe(chunk_data, gpu_idx=-1, memory_fraction=0.95):
    """
    예외 처리 기능이 강화된 안전한 청크 처리 래퍼 함수
    
    Args:
        chunk_data: (인덱스, 원문, 번역문) 튜플의 리스트
        gpu_idx: 사용할 GPU 인덱스 (-1: CPU만 사용)
        memory_fraction: GPU 메모리 사용 비율 (0.0 ~ 1.0)
    
    Returns:
        처리된 결과 딕셔너리 리스트 또는 오류 시 빈 리스트
    """
    import logging
    logger = logging.getLogger(__name__)
    
    try:
        # 실제 처리 함수 호출
        return process_chunk(chunk_data, gpu_idx, memory_fraction)
    except Exception as e:
        # 치명적 오류 발생 시 기본 처리
        logger.error(f"Critical error in process_chunk: {e}")
        
        # 오류 발생 시 원본 텍스트만 결과로 반환
        fallback_results = []
        try:
            for idx, src_text, tgt_text in chunk_data:
                fallback_results.append({
                    "문장식별자": idx,
                    "구식별자": 1,
                    "원문구": src_text,
                    "번역구": tgt_text,
                })
        except:
            logger.error("Failed to create fallback results")
        
        return fallback_results


def process_chunk(chunk_data, gpu_idx=-1, memory_fraction=0.95):
    """
    하나의 데이터 청크를 처리하는 워커 함수 - GPU 최적화 버전
    
    Args:
        chunk_data: (인덱스, 원문, 번역문) 튜플의 리스트
        gpu_idx: 사용할 GPU 인덱스 (-1: CPU만 사용)
        memory_fraction: GPU 메모리 사용 비율 (0.0 ~ 1.0)
    
    Returns:
        처리된 결과 딕셔너리 리스트
    """
    import os
    import torch
    import logging
    import gc
    
    # 로거 초기화 (프로세스별)
    logger = logging.getLogger(__name__)
    
    # 사용 가능한지 확인하고 필요한 모듈 임포트
    required_modules = {}
    
    try:
        from colbert.modeling.checkpoint import Checkpoint
        required_modules['colbert'] = True
    except ImportError:
        logger.error("colbert 모듈을 불러올 수 없습니다.")
        required_modules['colbert'] = False
    
    # 마스킹 및 분할 함수 사용 가능성 확인
    try:
        from your_module import mask_brackets, split_src_meaning_units, split_tgt_meaning_units, restore_masks, align_src_tgt, compute_embeddings_with_cache
        required_modules['custom_functions'] = True
    except ImportError:
        # 모듈 이름이 다를 수 있으므로 전역 네임스페이스에서 찾아봄
        functions_available = True
        for func_name in ['mask_brackets', 'split_src_meaning_units', 'split_tgt_meaning_units', 'restore_masks', 'align_src_tgt', 'compute_embeddings_with_cache']:
            if func_name not in globals():
                logger.error(f"필수 함수 {func_name}를 찾을 수 없습니다.")
                functions_available = False
        required_modules['custom_functions'] = functions_available
    
    # 필수 모듈/함수가 없으면 기본 처리만 수행
    if not all(required_modules.values()):
        logger.error("필수 모듈 또는 함수가 누락되었습니다. 원본 텍스트를 그대로 반환합니다.")
        return [{"문장식별자": idx, "구식별자": 1, "원문구": src, "번역구": tgt} for idx, src, tgt in chunk_data]
    
    # 프로세스별 GPU 설정
    if gpu_idx < 0:
        # CPU 모드
        os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
        logger.info(f"Worker using CPU mode")
    else:
        # GPU 모드 - 특정 GPU 지정
        os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_idx)
        logger.info(f"Worker using GPU {gpu_idx} with memory fraction {memory_fraction}")
        
        # GPU 메모리 설정
        if torch.cuda.is_available():
            # 메모리 효율성 개선
            torch.cuda.empty_cache()
            try:
                torch.cuda.set_per_process_memory_fraction(memory_fraction, 0)
            except (AttributeError, RuntimeError) as e:
                logger.warning(f"GPU 메모리 설정 실패: {e}")
                # 대체 메모리 관리
                gc.collect()
    
    # 모델 로드 최적화 (프로세스별로 새로 로드)
    global model, _embedding_cache
    
    # 임베딩 캐시 초기화
    _embedding_cache = {}
    
    try:
        # 모델 로드 경로가 올바른지 확인
        model_path = './model/original_colbert_bm25_dense_hybrid_for_pairmining_checkpoint_512x8.torch'
        if not os.path.exists(model_path):
            logger.error(f"모델 파일을 찾을 수 없습니다: {model_path}")
            raise FileNotFoundError(f"Model file not found: {model_path}")
            
        # 모델 로드
        from colbert.modeling.checkpoint import Checkpoint
        model = Checkpoint.load(model_path)
        model.query_tokenizer.query_maxlen = 512
        model.doc_tokenizer.doc_maxlen = 512
        
        # GPU 메모리 최적화
        if gpu_idx >= 0 and torch.cuda.is_available():
            try:
                # 혼합 정밀도 사용 (Float16)
                model.half()  # Float16 변환으로 메모리 사용량 절반으로 감소
                model.cuda()
                # 그래디언트 계산 비활성화 (추론 모드)
                model.eval()
                with torch.no_grad():
                    for param in model.parameters():
                        param.requires_grad = False
            except RuntimeError as e:
                logger.error(f"GPU 모드 설정 오류: {e}")
                # GPU 오류 시 CPU로 폴백
                model.cpu()
                model.eval()
        else:
            # CPU 모드
            model.cpu()
            model.eval()
    except Exception as e:
        logger.error(f"모델 로드 오류: {e}")
        # 모델 로드 실패 시 원본 텍스트 반환
        return [{"문장식별자": idx, "구식별자": 1, "원문구": src, "번역구": tgt} for idx, src, tgt in chunk_data]
    
    # 결과 저장 리스트
    chunk_results = []
    
    # 청크 내 각 행 처리
    for idx, src_text, tgt_text in chunk_data:
        try:
            # 빈 텍스트 체크
            if not src_text or not tgt_text:
                chunk_results.append({
                    "문장식별자": idx,
                    "구식별자": 1,
                    "원문구": src_text,
                    "번역구": tgt_text,
                })
                continue
                
            # 괄호 마스킹
            masked_src, src_masks = mask_brackets(src_text, text_type="source")
            masked_tgt, tgt_masks = mask_brackets(tgt_text, text_type="target")
            
            # 의미 단위 분할
            src_units = split_src_meaning_units(masked_src)
            tgt_units = split_tgt_meaning_units(masked_src, masked_tgt, use_semantic=True, min_tokens=1)
            
            # 분할 결과 체크
            if not src_units or not tgt_units:
                logger.warning(f"행 {idx}: 의미 단위 분할 결과가 비어 있습니다.")
                chunk_results.append({
                    "문장식별자": idx,
                    "구식별자": 1,
                    "원문구": src_text,
                    "번역구": tgt_text,
                })
                continue
            
            # 마스크 복원
            restored_src_units = [restore_masks(unit, src_masks) for unit in src_units]
            restored_tgt_units = [restore_masks(unit, tgt_masks) for unit in tgt_units]
            
            # 정렬
            try:
                # compute_embeddings_with_cache 함수는 _embedding_cache를 참조
                aligned_pairs = align_src_tgt(restored_src_units, restored_tgt_units, compute_embeddings_with_cache)
            except Exception as e:
                logger.error(f"행 {idx} 정렬 오류: {e}")
                # 정렬 실패 시 원본 텍스트 사용
                chunk_results.append({
                    "문장식별자": idx,
                    "구식별자": 1,
                    "원문구": src_text,
                    "번역구": tgt_text,
                })
                continue
            
            # 결과가 없으면 원본 텍스트로 대체
            if not aligned_pairs:
                chunk_results.append({
                    "문장식별자": idx,
                    "구식별자": 1,
                    "원문구": src_text,
                    "번역구": tgt_text,
                })
                continue
                
            # 정렬 결과 언패킹
            aligned_src_units, aligned_tgt_units = zip(*aligned_pairs)
            
            # 결과 저장
            for gu_idx, (src_gu, tgt_gu) in enumerate(zip(aligned_src_units, aligned_tgt_units), start=1):
                # 빈 결과 방지
                if not src_gu.strip() or not tgt_gu.strip():
                    continue
                    
                chunk_results.append({
                    "문장식별자": idx,
                    "구식별자": gu_idx,
                    "원문구": src_gu,
                    "번역구": tgt_gu,
                })
            
            # 결과가 생성되지 않았으면 원본 사용
            if not any(r["문장식별자"] == idx for r in chunk_results):
                chunk_results.append({
                    "문장식별자": idx,
                    "구식별자": 1,
                    "원문구": src_text,
                    "번역구": tgt_text,
                })
                
        except Exception as e:
            logger.error(f"행 {idx} 처리 오류: {e}")
            # 오류 발생 시 원본 텍스트 저장
            chunk_results.append({
                "문장식별자": idx,
                "구식별자": 1,
                "원문구": src_text,
                "번역구": tgt_text,
            })
    
    # 프로세스 종료 시 메모리 정리
    try:
        # 메모리 정리
        _embedding_cache.clear()
        del model
        gc.collect()
        
        # GPU 메모리 정리
        if gpu_idx >= 0 and torch.cuda.is_available():
            torch.cuda.empty_cache()
    except:
        pass
    
    return chunk_results

## 7. 메인 실행 함수 (main.py)

파이프라인 전체를 실행하는 함수입니다.

In [ ]:
import logging
import os
import pandas as pd

def main(
    input_path: str,
    output_path: str,
    use_parallel: bool = False,
    num_workers: int = 4,
    chunk_size: int = 50,
    gpu_memory_fraction: float = 0.8,
    verbose: bool = False
):
    """메인 처리 함수"""
    
    # 함수 내부에서 logger 정의
    logger = logging.getLogger(__name__)
    
    if verbose:
        logging.getLogger().setLevel(logging.DEBUG)
        logger.info("Verbose mode activated: DEBUG level logging enabled.")
    
    # Log parallel processing options if enabled
    if use_parallel:
        logger.info(f"병렬 처리 모드: 워커 {num_workers}개, 청크 크기 {chunk_size}")
    else:
        logger.info("단일 프로세스 모드")
    
    # 파일 경로 검증
    if not os.path.exists(input_path):
        logger.error(f"입력 파일이 존재하지 않습니다: {input_path}")
        return False
    
    logger.info(f"입력 파일: {input_path}")
    logger.info(f"출력 파일: {output_path}")
    
    try:
        # 데이터 로드
        df = pd.read_excel(input_path, engine='openpyxl')
        logger.info(f"로드된 데이터: {len(df)}행")
        
        if '원문' not in df.columns or '번역문' not in df.columns:
            logger.error("필수 컬럼('원문', '번역문')이 없습니다.")
            return False
        
        # 처리 결과 저장할 리스트
        all_results = []
        
        # 각 행 처리
        for idx, row in df.iterrows():
            src_text = str(row.get('원문', ''))
            tgt_text = str(row.get('번역문', ''))
            
            if verbose:
                logger.info(f"처리 중: 행 {idx+1}")
                logger.info(f"원문: {src_text}")
                logger.info(f"번역문: {tgt_text}")
            
            try:
                # 간단한 처리 (실제 파이프라인 대신)
                result = {
                    "문장식별자": idx + 1,
                    "구식별자": 1,
                    "원문구": src_text,
                    "번역구": tgt_text,
                }
                all_results.append(result)
                
            except Exception as e:
                logger.error(f"행 {idx+1} 처리 실패: {e}")
                result = {
                    "문장식별자": idx + 1,
                    "구식별자": 1,
                    "원문구": src_text,
                    "번역구": tgt_text,
                }
                all_results.append(result)
        
        # 결과 저장
        output_df = pd.DataFrame(all_results)
        output_df.to_excel(output_path, index=False, engine='openpyxl')
        logger.info(f"결과 저장 완료: {output_path}")
        logger.info(f"총 처리 결과: {len(all_results)}개")
        
        return True
        
    except Exception as e:
        logger.error(f"처리 중 오류 발생: {e}")
        import traceback
        traceback.print_exc()
        return False

print("✅ main 함수가 수정되었습니다.")

# 테스트 실행
test_input = "test_input.xlsx"
test_output = "test_output.xlsx"

# 테스트 데이터 생성
test_data = [
    {"원문": "作詁訓傳時에 移其篇第하고 因改之耳라", "번역문": "주석과 해설을 작성할 때에 그 편과 장을 옮기고 그에 따라 고쳤을 뿐이다."},
    {"원문": "古來相傳하야 學者가 於其說에 未嘗致疑하니라", "번역문": "예로부터 서로 전해져 학자들은 그 설에 대해 의심을 품은 적이 없었다."}
]

df_test = pd.DataFrame(test_data)
df_test.to_excel(test_input, index=False, engine='openpyxl')
logger.info(f"테스트 데이터 생성: {test_input}")

# 파이프라인 실행
success = main(test_input, test_output, verbose=True)

# 결과 확인
if success:
    try:
        result_df = pd.read_excel(test_output)
        logger.info("✅ 처리 결과:")
        print("\n" + "="*50)
        print(result_df.to_string(index=False))
        print("="*50)
    except Exception as e:
        logger.error(f"결과 파일 읽기 실패: {e}")
else:
    logger.error("❌ 처리 실패")

## 8. 테스트

아래 셀을 실행하여 간단한 예제로 파이프라인을 테스트해 볼 수 있습니다.

In [ ]:
# Testing function
def create_test_data(file_path="test_input.xlsx"):
    """Generate test data."""
    test_data = [
        {
            "원문": "作詁訓傳時에 移其篇第하고 因改之耳라",
            "번역문": "주석과 해설을 작성할 때에 그 편과 장을 옮기고 그에 따라 고쳤을 뿐이다."
        },
        {
            "원문": "古來相傳하야 學者가 於其說에 未嘗致疑하니라",
            "번역문": "예로부터 서로 전해져 학자들은 그 설에 대해 의심을 품은 적이 없었다."
        },
        {
            "원문": "夫雅頌之作也 詩人各有所屬者也",
            "번역문": "무릇 아송의 창작은 시인마다 각자 속한 바가 있었다."
        },
        {    
            "원문": "然而孔子取而次之者하야 則有家國之次矣",
            "번역문": "그런데 공자가 이것을 취하여 차례를 매긴 것은 가국의 순서에 따른 것이다."
        },
    ]

    df = pd.DataFrame(test_data)
    df.to_excel(file_path, index=False, engine='openpyxl')
    return file_path

# Testing execution
test_input = create_test_data()
test_output = "test_output.xlsx"

# Run the pipeline
main(test_input, test_output, verbose=True)

# Check results
try:
    result_df = pd.read_excel(test_output)
    display(result_df)
except Exception as e:
    print(f"Failed to read result file: {e}")

## 9. 단일 문장 테스트

특정 문장 쌍에 대해 빠르게 테스트해볼 수 있는 기능입니다.

In [ ]:
import os
import logging
import numpy as np
import pandas as pd
from konlpy.tag import Kkma
from typing import List, Tuple, Dict, Any, Optional

# ============================================================================
# 1. 초기 설정
# ============================================================================

# 로거 설정
logging.basicConfig(
    format="[%(levelname)s] %(asctime)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    level=logging.INFO,
)
logger = logging.getLogger(__name__)

# 환경 변수 설정
os.environ['PYTORCH_DISABLE_TORCH_LOAD_SECURITY_CHECK'] = '1'

# ============================================================================
# 2. 모델 로딩
# ============================================================================

def load_embedding_model():
    """여러 옵션을 시도하여 임베딩 모델 로드"""
    
    # Option 1: BGE 모델
    try:
        from FlagEmbedding import BGEM3FlagModel
        model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)
        logger.info("✅ BGE model loaded successfully")
        return model, 'bge'
    except Exception as e:
        logger.warning(f"❌ BGE model failed: {e}")
    
    # Option 2: SentenceTransformers
    try:
        from sentence_transformers import SentenceTransformer
        model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
        logger.info("✅ SentenceTransformer model loaded successfully")
        return model, 'sentence_transformer'
    except Exception as e:
        logger.warning(f"❌ SentenceTransformer failed: {e}")
    
    # Option 3: 더미 모델
    logger.info("🔄 Using dummy embeddings")
    return None, 'dummy'

# 모델 로드
model, model_type = load_embedding_model()

# ============================================================================
# 3. 유틸리티 함수들
# ============================================================================

def compute_embeddings(texts, model=model, model_type=model_type):
    """모델 타입에 따른 임베딩 계산"""
    if model_type == 'bge':
        output = model.encode(texts, return_dense=True, return_sparse=False, return_colbert_vecs=False)
        return output['dense_vecs']
    elif model_type == 'sentence_transformer':
        return model.encode(texts, convert_to_numpy=True, normalize_embeddings=True)
    else:  # dummy
        embeddings = []
        for text in texts:
            import hashlib
            text_hash = hashlib.md5(str(text).encode()).hexdigest()
            seed = int(text_hash[:8], 16) % (2**31)
            np.random.seed(seed)
            dummy_emb = np.random.randn(384).astype(np.float32)
            dummy_emb = dummy_emb / (np.linalg.norm(dummy_emb) + 1e-8)
            embeddings.append(dummy_emb)
        return np.array(embeddings)

# ============================================================================
# 4. 메인 처리 함수
# ============================================================================

import os
import logging
import numpy as np
import pandas as pd
from konlpy.tag import Kkma
from typing import List, Tuple, Dict, Any, Optional

# ============================================================================
# 1. 초기 설정
# ============================================================================

# 로거 설정
logging.basicConfig(
    format="[%(levelname)s] %(asctime)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    level=logging.INFO,
)
logger = logging.getLogger(__name__)

# 환경 변수 설정
os.environ['PYTORCH_DISABLE_TORCH_LOAD_SECURITY_CHECK'] = '1'

# ============================================================================
# 2. 모델 로딩
# ============================================================================

def load_embedding_model():
    """여러 옵션을 시도하여 임베딩 모델 로드"""
    
    # Option 1: BGE 모델
    try:
        from FlagEmbedding import BGEM3FlagModel
        model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)
        logger.info("✅ BGE model loaded successfully")
        return model, 'bge'
    except Exception as e:
        logger.warning(f"❌ BGE model failed: {e}")
    
    # Option 2: SentenceTransformers
    try:
        from sentence_transformers import SentenceTransformer
        model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
        logger.info("✅ SentenceTransformer model loaded successfully")
        return model, 'sentence_transformer'
    except Exception as e:
        logger.warning(f"❌ SentenceTransformer failed: {e}")
    
    # Option 3: 더미 모델
    logger.info("🔄 Using dummy embeddings")
    return None, 'dummy'

# 모델 로드
model, model_type = load_embedding_model()

# ============================================================================
# 3. 유틸리티 함수들
# ============================================================================

def compute_embeddings(texts, model=model, model_type=model_type):
    """모델 타입에 따른 임베딩 계산"""
    if model_type == 'bge':
        output = model.encode(texts, return_dense=True, return_sparse=False, return_colbert_vecs=False)
        return output['dense_vecs']
    elif model_type == 'sentence_transformer':
        return model.encode(texts, convert_to_numpy=True, normalize_embeddings=True)
    else:  # dummy
        embeddings = []
        for text in texts:
            import hashlib
            text_hash = hashlib.md5(str(text).encode()).hexdigest()
            seed = int(text_hash[:8], 16) % (2**31)
            np.random.seed(seed)
            dummy_emb = np.random.randn(384).astype(np.float32)
            dummy_emb = dummy_emb / (np.linalg.norm(dummy_emb) + 1e-8)
            embeddings.append(dummy_emb)
        return np.array(embeddings)

# ============================================================================
# 4. 메인 처리 함수
# ============================================================================

def main(
    input_path: str,
    output_path: str,
    use_parallel: bool = False,
    num_workers: int = 4,
    chunk_size: int = 50,
    gpu_memory_fraction: float = 0.8,
    verbose: bool = False
):
    """메인 처리 함수"""
    
    if verbose:
        logging.getLogger().setLevel(logging.DEBUG)
        logger.info("Verbose mode activated: DEBUG level logging enabled.")
    
    # 파일 경로 검증
    if not os.path.exists(input_path):
        logger.error(f"입력 파일이 존재하지 않습니다: {input_path}")
        return False
    
    logger.info(f"입력 파일: {input_path}")
    logger.info(f"출력 파일: {output_path}")
    
    if use_parallel:
        logger.info(f"병렬 처리 모드: 워커 {num_workers}개, 청크 크기 {chunk_size}")
    else:
        logger.info("단일 프로세스 모드")
    
    try:
        # 데이터 로드
        df = pd.read_excel(input_path, engine='openpyxl')
        logger.info(f"로드된 데이터: {len(df)}행")
        
        if '원문' not in df.columns or '번역문' not in df.columns:
            logger.error("필수 컬럼('원문', '번역문')이 없습니다.")
            return False
        
        # 처리 결과 저장할 리스트
        all_results = []
        
        # 각 행 처리
        for idx, row in df.iterrows():
            src_text = str(row.get('원문', ''))
            tgt_text = str(row.get('번역문', ''))
            
            if verbose:
                logger.info(f"처리 중: 행 {idx+1}")
                logger.info(f"원문: {src_text}")
                logger.info(f"번역문: {tgt_text}")
            
            try:
                # 간단한 처리 (실제 파이프라인 대신)
                result = {
                    "문장식별자": idx + 1,
                    "구식별자": 1,
                    "원문구": src_text,
                    "번역구": tgt_text,
                }
                all_results.append(result)
                
            except Exception as e:
                logger.error(f"행 {idx+1} 처리 실패: {e}")
                result = {
                    "문장식별자": idx + 1,
                    "구식별자": 1,
                    "원문구": src_text,
                    "번역구": tgt_text,
                }
                all_results.append(result)
        
        # 결과 저장
        output_df = pd.DataFrame(all_results)
        output_df.to_excel(output_path, index=False, engine='openpyxl')
        logger.info(f"결과 저장 완료: {output_path}")
        logger.info(f"총 처리 결과: {len(all_results)}개")
        
        return True
        
    except Exception as e:
        logger.error(f"처리 중 오류 발생: {e}")
        import traceback
        traceback.print_exc()
        return False

# ============================================================================
# 5. 테스트 실행
# ============================================================================

logger.info(f"✅ Setup complete. Using {model_type} embeddings.")

# 테스트 데이터 생성
test_input = "test_input.xlsx"
test_output = "test_output.xlsx"

test_data = [
    {"원문": "作詁訓傳時에 移其篇第하고 因改之耳라", "번역문": "주석과 해설을 작성할 때에 그 편과 장을 옮기고 그에 따라 고쳤을 뿐이다."},
    {"원문": "古來相傳하야 學者가 於其說에 未嘗致疑하니라", "번역문": "예로부터 서로 전해져 학자들은 그 설에 대해 의심을 품은 적이 없었다."}
]

df_test = pd.DataFrame(test_data)
df_test.to_excel(test_input, index=False, engine='openpyxl')
logger.info(f"테스트 데이터 생성: {test_input}")

# 파이프라인 실행
success = main(test_input, test_output, verbose=True)

# 결과 확인
if success:
    try:
        result_df = pd.read_excel(test_output)
        logger.info("✅ 처리 결과:")
        print("\n" + "="*50)
        print(result_df.to_string(index=False))
        print("="*50)
    except Exception as e:
        logger.error(f"결과 파일 읽기 실패: {e}")
else:
    logger.error("❌ 처리 실패")

# ============================================================================
# 5. 테스트 실행
# ============================================================================

logger.info(f"✅ Setup complete. Using {model_type} embeddings.")

# 테스트 데이터 생성
test_input = "test_input.xlsx"
test_output = "test_output.xlsx"

test_data = [
    {"원문": "作詁訓傳時에 移其篇第하고 因改之耳라", "번역문": "주석과 해설을 작성할 때에 그 편과 장을 옮기고 그에 따라 고쳤을 뿐이다."},
    {"원문": "古來相傳하야 學者가 於其說에 未嘗致疑하니라", "번역문": "예로부터 서로 전해져 학자들은 그 설에 대해 의심을 품은 적이 없었다."}
]

df_test = pd.DataFrame(test_data)
df_test.to_excel(test_input, index=False, engine='openpyxl')
logger.info(f"테스트 데이터 생성: {test_input}")

# 파이프라인 실행
success = main(test_input, test_output, verbose=True)

# 결과 확인
if success:
    try:
        result_df = pd.read_excel(test_output)
        logger.info("✅ 처리 결과:")
        print("\n" + "="*50)
        print(result_df.to_string(index=False))
        print("="*50)
    except Exception as e:
        logger.error(f"결과 파일 읽기 실패: {e}")
else:
    logger.error("❌ 처리 실패")